In [1]:
# !pip install -q openai
# !pip install -q langchain
# !pip install -q guardrails-ai
# !pip install -q faiss-cpu
# !pip install -q pypdf
# !pip install -q python-dotenv
# !pip install -q datasets
# !pip install -q huggingface_hub

In [2]:
import os
from dotenv import load_dotenv
import json

from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage,  SystemMessage

#Guardrails
import openai
from rich import print
from langchain.output_parsers import GuardrailsOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

## Load Environment Varible

In [3]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")
huggingface_api_key = os.environ.get("HUGGINGFACE_API_KEY")

## Loading the Document

In [4]:
# loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
loader = PyPDFDirectoryLoader("../data")
# loader = PyPDFDirectoryLoader("../cyber")
data = loader.load()

In [5]:
print(data[0].page_content[:100])

© 2020 . D
r. Zhi Gang Sha  & Rulin Xiu . This is a research/review paper, distributed under the ter

In [6]:
file_name = "Extracted_data.txt"

# Open the file in write mode
with open(file_name, "w",encoding="utf-8") as file:
    for item in data:
        # Extract the "page_content" attribute from each item in the list
        page_content = item.page_content

        # Write the page_content to the file followed by a newline
        file.write(page_content + "\n")

print(f"Data has been written to {file_name}")

Data has been written to Extracted_data.txt

Chunking the Documents

In [7]:
#Step 05: Split the Extracted Data into Text Chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=500)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=500)
text_chunks = text_splitter.split_documents(data)
print("Lenth of the whole documentation is:",len(text_chunks))

Lenth of the whole documentation is: 364

## Prompt template

[INPUT TEXT]

[CONTEXT ]

[REQUEST FOR Q+A]

[RESPONSE SAMPLE]


Getting Context of the Data

In [8]:
chat = ChatOpenAI(temperature=0.6) # for sythetic data generation
model = OpenAI(temperature=0) # for parsing the output

In [9]:
messages = [
    HumanMessage(
        content=f"{data[0].page_content}give the above information give me a single line rich summary with keywords in the beginning which can be used to descibe the entiner inforamtion"
    ),
]
response = chat(messages)

print(response.content)

This research paper proposes Information Mechanics as a framework to understand the process of "It from Bit," where
information determines observed phenomena. It introduces information space and time as the basic elements and 
suggests that their interaction creates everything we observe. The paper derives an information action and function
to calculate the maximum possibilities and possible states in a system. Keywords: information mechanics, hierarchy 
problem, cosmological constant problem, dark matter, dark energy, black hole, it from bit, grand unification 
theory.

## Generating QnA 

- *prompt* : provide {questions_per_chunk} question and answer pairs base on the text above , The Question must begin with \n"In the context of ...\".The answer borrow, verbatim, from the text above. In providing each question consider that the reader does not see of have access to any of the other questions from context. Vary the style and formate fo quesitons. Respond in plain test on a new line for each question and answer. Do not include Do no include qestion numbers, Here is an exmaple of two question and answer paids:\n\n {tain_sample}

In [10]:
dataset_name = "TaoGPT-v1.json"

json_response_format = [
                {
                    "question": "In the context of ...",
                    "answer": "..."
                },
                {
                    "question": "In the context of ...",
                    "answer": "..."
                },
                {
                    "question": "In the context of ...",
                    "answer": "..."
                }
            ]

rail_spec = """
<rail version="0.1">
<output>
<list name="data" description="list of question answer pairs">
    <object>
        <string name="question" description="the question"/>
        <string name="answer"  description="the answer"/>
    </object>
</list>
</output>
<prompt>

Given the following list of json question and answer paids , please extract it in a proper JSON formate 
${generated_data_json}

${gr.complete_json_suffix_v2}
</prompt>
</rail>
"""
def write_to_json_file(json_data, json_file_name):
    try:
        try:
            with open(json_file_name, "r") as outfile:
                data = json.load(outfile)
            for json_data_pairs in json_data:
                data.append(json_data_pairs)
            with open(json_file_name, "w") as outfile:
                json.dump(data, outfile)
        except FileNotFoundError:
            with open(json_file_name, "w") as outfile:
                json.dump(json_data, outfile)        
    except Exception as e:
        print("Error in write_to_json_file", e)

# def write_to_json_file(json_data_list, json_file_name):
#   try:
#     with open(json_file_name, "r") as outfile:
#       data = json.load(outfile)
    
#     for json_data in json_data_list:
#       data.append(json_data)

#     with open(json_file_name, "w") as outfile: 
#       json.dump(data, outfile)

#   except FileNotFoundError:
#     data = json_data_list  
#     with open(json_file_name, "w") as outfile:
#       json.dump(data, outfile)

#   except Exception as e:
#     print("Error in write_to_json_file", e)

In [11]:
for text in text_chunks:
    try:
        context_messages = [
            HumanMessage(
                content=f"{text.page_content}give the above information give me a single line rich summary with keywords in the beginning which can be used to descibe the entiner inforamtion"
            ),
        ]
        context_response = chat(context_messages)
        print(context_response.content)
        try:
            generated_messages = [
                HumanMessage(
                    content=f"""given the context:{context_response} 
                    and the information {text.page_content} provide 5 question and answer pairs base on the text above , 
                    The Question must begin with 
                    "In the context of ...\".The answer borrow, verbatim, from the text above. In providing each question consider that the reader does not see of have access to any of the other questions from context. Vary the style and formate fo quesitons. 
                    Respond in only JSON following this formate
                    {json_response_format} and nothing else"""
                ),
            ]
            generated_response = chat(generated_messages)
            try:
                eval_generated_json = eval(generated_response.content)
                if len(eval_generated_json) >= 1:
                    write_to_json_file(eval_generated_json, dataset_name)
                    print(eval_generated_json)
                else:
                    raise Exception("Data generated not in the right format")
            except:
                # output_parser = GuardrailsOutputParser.from_rail_string(extractor_rail)
                output_parser = GuardrailsOutputParser.from_rail_string(rail_spec, api=openai.ChatCompletion.create)
                prompt = PromptTemplate(
                    template=output_parser.guard.base_prompt,
                    input_variables=output_parser.guard.prompt.variable_names,
                )

                model = OpenAI(temperature=0) # type: ignore
                generated_response_final = model(prompt.format_prompt(generated_data_json=generated_response.content).to_string())
                generated_response_final = output_parser.parse(generated_response_final)
                generated_response_final = generated_response_final["data"]
                print(generated_response_final)                
                write_to_json_file(generated_response_final, dataset_name)
        except:
            print("Error in generating the data")
    except Exception as e:
        print("Failed to get context of the text")
    print("\n---------------------------------------------------\n")
    


Summary: This research paper proposes Information Mechanics as a framework to understand the process of "It from 
Bit," where information gives rise to physics. The authors suggest that information determines observed phenomena, 
and the interaction of information space and time creates everything we observe. They derive an action, the 
information action, and the information function to represent the maximum possibilities of information in a system.
This paper provides insights into the dynamics of information and its role in shaping our reality.

Keywords: Information Mechanics, It from Bit, observed phenomena, information space, information time, information 
action, information function.

[
    {
        'question': 'In the context of the research paper, what does "It from Bit" refer to?',
        'answer': 'information gives rise to physics'
    },
    {
        'question': 'In the context of the research paper, what is proposed as a framework to understand "It from 
Bit"?',
        'answer': 'Information Mechanics'
    },
    {
        'question': 'In the context of the research paper, what are the two basic elements that make up 
information?',
        'answer': 'information space and time'
    },
    {
        'question': 'In the context of the research paper, what is derived as the maximum possibilities of 
information in a system?',
        'answer': 'the information action'
    },
    {
        'question': 'In the context of the research paper, what role does information play in shaping our 
reality?',
        'answer': 'it determines observed phenomena'
    }
]

---------------------------------------------------

The interaction of information space and time creates everything we observe, leading to the development of 
information mechanics. This theory addresses various phenomena such as the hierarchy problem, cosmological constant
problem, dark matter, dark energy, and black holes. The concept of "it from bit" suggests that information is the 
fundamental building block of the universe. Keywords: information mechanics, hierarchy problem, cosmological 
constant problem, dark matter, dark energy, black hole, it from bit, grand unification theory.

[
    {
        'question': 'In the context of information mechanics, what does the concept of "it from bit" suggest?',
        'answer': 'The concept of "it from bit" suggests that information is the fundamental building block of the 
universe.'
    },
    {
        'question': 'In the context of information mechanics, what phenomena does the theory address?',
        'answer': 'The theory addresses various phenomena such as the hierarchy problem, cosmological constant 
problem, dark matter, dark energy, and black holes.'
    },
    {
        'question': 'In the context of information mechanics, what are the two basic elements making up 
information?',
        'answer': 'The two basic elements making up information are information space and time.'
    },
    {
        'question': 'In the context of information mechanics, what does the information action represent?',
        'answer': 'The information action represents the maximum possibilities, i.e., information, in a system.'
    },
    {
        'question': 'In the context of information mechanics, what does the information function calculate?',
        'answer': 'The information function calculates the possible states in a system.'
    }
]

...

---------------------------------------------------

The information provided includes a mathematical theory of communication, pseudoscience and the paranormal, and the
composition of existence and the definition of spirit, heart, and mind according to quantum physics.

Error in write_to_json_file Expecting value: line 1 column 357232 (char 357231)

[
    {
        'question': 'In the context of the information provided, what does the content include?',
        'answer': 'The information provided includes a mathematical theory of communication, pseudoscience and the 
paranormal, and the composition of existence and the definition of spirit, heart, and mind according to quantum 
physics.'
    },
    {
        'question': 'In the context of the information provided, what is the title and author of the paper?',
        'answer': 'The title of the paper is "A mathematical theory of communication" and the author is C. E. 
Shannon.'
    },
    {
        'question': 'In the context of the information provided, what is the title and author of the book?',
        'answer': 'The title of the book is "Pseudoscience and the Paranormal" and the author is Terence Hines.'
    },
    {
        'question': 'In the context of the information provided, what is the title and author of the book on 
mathematical foundations of quantum mechanics?',
        'answer': 'The title of the book is "Mathematical Foundations of Quantum Mechanics" and the author is J. 
Von Neumann.'
    },
    {
        'question': 'In the context of the information provided, what is the name of the article and its 
publication details?',
        'answer': 'The name of the article is "A mathematical theory of communication" and it was published in the 
Bell System Technical Journal, vol. 27, pp. 379–423 and 623–656, in July and October 1948.'
    },
    {
        'question': 'In the context of the information provided, who are the authors of the composition of 
existence and the definition of spirit, heart, and mind according to quantum physics?',
        'answer': 'The authors are Zhi Gang Sha and Rulin Xiu.'
    }
]

---------------------------------------------------

## HuggingFace

In [13]:
from huggingface_hub import notebook_login
from huggingface_hub import HfApi
from datasets import load_dataset
api = HfApi(token=huggingface_api_key)


In [15]:
dataset = load_dataset('json', data_files="./TaoGPT-v1.json")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/Adithya/.cache/huggingface/datasets/json/default-ce654e8b2b1176ad/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1554
    })
})

In [16]:
# dataset.push_to_hub("Dataset name")
dataset.push_to_hub("agency888/TaoGPT-v1")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
# Pushing Dataset.json
api.upload_file(
    path_or_fileobj="./TaoGPT-v1.json",
    path_in_repo="TaoGPT-v1.json",
    repo_id="agency888/TaoGPT-v1",
    repo_type="dataset",
)

'https://huggingface.co/datasets/agency888/TaoGPT-v1/blob/main/TaoGPT-v1.json'

In [18]:
# Pushing Dataset Readme
api.upload_file(
    path_or_fileobj="./README.md",
    path_in_repo="README.md",
    repo_id="agency888/TaoGPT-v1",
    repo_type="dataset",
)

'https://huggingface.co/datasets/agency888/TaoGPT-v1/blob/main/README.md'